## Checkpoints
- [ ] ``load -> save -> compare``
- [ ] ``make pkg from scratch``
- [ ] ``layers generator``

In [142]:
upgrade_py = True
if upgrade_py:
    !python3.10 -m pip install --upgrade pip
    !pip install graph-notebook
    !pip install graphviz
    !pip install pydot
    !pip install networkx
    !pip install pyvis
    !pip install platform
    !pip install hashlib
    !pip install jycm
    !pip install pipreqs
%load_ext line_profiler

ERROR: Could not find a version that satisfies the requirement platform (from versions: none)
ERROR: No matching distribution found for platform
ERROR: Ignored the following yanked versions: 20081119
ERROR: Could not find a version that satisfies the requirement hashlib (from versions: none)
ERROR: No matching distribution found for hashlib
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [143]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [144]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [145]:
import os
import sys
import matplotlib.pyplot as plt
import networkx as nx
import json
import pprint
import numpy as np
import pandas as pd
import time
import platform
import hashlib
from enum import Enum
from pyvis.network import Network
from IPython.display import display, HTML
import copy
from jycm.jycm import YouchamaJsonDiffer

from pathlib import Path
os.environ["MGENN_DEBUG"] = "Y"


In [146]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())

import mcore as mc

from common import *

In [147]:
pkg = mc.Package()
pkg.loadFile('data/namespacetest_rev0.pkg')
pkg.counts()


[2024-12-27 14:24:13]in[counts.254] inputs:9, outputs:10, neurons:10, links:10, total:39


In [148]:
def showGraph(pkg):
    #net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    nt = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, directed =True)

    ZH = "ZERO_HOLE"
    def hasNode(nodes, key)->bool:
        for node in nx_graph.nodes:
            if str(node) == str(key):
                return True
        return False
    nx_graph = nx.DiGraph()

    for n in pkg.neurons:
        nx_graph.add_node(str(n["id"]), size = 10 + int(float(n["currentEnergy"])), title=('neuron %d' % n["id"]), group=1)
    for o in pkg.outputs:
        nx_graph.add_node(str(o["id"]), size = 10, title=('outputs %d' % n["id"]), group=2)
    for i in pkg.inputs:
        nx_graph.add_node(str(i["name"]), size = 5, title=('input %s' % i["name"]), group=3)

    #make direct links
    nx_graph.add_node(str(ZH), size = 1, title=("missed node"), group=1)
    l = pkg.links[1]
    start, stop = pkg.linkEnds(l)
    #print("link ", start, " to " , stop)
    #nx_graph.add_edge(str(start), str(stop))

    for l in pkg.links:
        print(l)
        start, stop = pkg.linkEnds(l)
        link_len = int(l['length'])
        print("link ", start, " to " , str(stop), " sz:", link_len)
        if not start:
            continue
            nx_graph.add_edge(ZH, stop, length = (link_len + 1), weight=(link_len + 1))
        if not hasNode(nx_graph.nodes, str(stop)):
            continue
            nx_graph.add_edge(ZH, stop, value = (link_len + 1), weight=(link_len + 1))
        nx_graph.add_edge(str(start), str(stop), length = (link_len + 1), weight=(link_len + 1) )

    for n in pkg.neurons:
        for rc in n['receivers']:
            if not pkg.isLink(rc):
                nx_graph.add_edge(str(n["id"]), str(rc), length = 1, weight = 1)
    nt.from_nx(nx_graph)
    return nt.show('nx.html')

In [149]:
showGraph(pkg)

{'attenuationPerTick': '0.020000', 'events': [], 'id': 10, 'length': 2, 'receiverId': 64}
[2024-12-27 14:24:14]in[linkEnds.287] link 10 is hanging down
link  0  to  64  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 11, 'length': 2, 'receiverId': 65}
link  Alias1  to  65  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 12, 'length': 2, 'receiverId': 66}
link  Alias2  to  66  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 13, 'length': 2, 'receiverId': 67}
link  Alias3  to  67  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 14, 'length': 2, 'receiverId': 68}
link  Alias4  to  68  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 15, 'length': 2, 'receiverId': 69}
link  Alias5  to  69  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 16, 'length': 2, 'receiverId': 70}
link  Alias6  to  70  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 17, 'length': 2, 'receiverId': 71}
link  Alias7  to  71  sz: 2

## low-level generators

In [150]:


p = pkg.clone()
n1 = ObjectFactory.makeNeuronData(peak = 10.0, receivers = [], leak = 0.1, currentEnergy = 3.0, id = p.nextId())
p.neurons.append(n1)
p.connect(n1["id"], 64)
p.connect(n1["id"], 65)
p.connect(n1["id"], 69)
g = showGraph(p)
print("new object",n1["id"])
g
pkg = None

[2024-12-27 14:24:15]in[connect.307] linked N[110] to 64
[2024-12-27 14:24:15]in[connect.307] linked N[110] to 65
[2024-12-27 14:24:15]in[connect.307] linked N[110] to 69
{'attenuationPerTick': '0.020000', 'events': [], 'id': 10, 'length': 2, 'receiverId': 64}
[2024-12-27 14:24:15]in[linkEnds.287] link 10 is hanging down
link  0  to  64  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 11, 'length': 2, 'receiverId': 65}
link  Alias1  to  65  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 12, 'length': 2, 'receiverId': 66}
link  Alias2  to  66  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 13, 'length': 2, 'receiverId': 67}
link  Alias3  to  67  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 14, 'length': 2, 'receiverId': 68}
link  Alias4  to  68  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 15, 'length': 2, 'receiverId': 69}
link  Alias5  to  69  sz: 2
{'attenuationPerTick': '0.020000', 'events': [], 'id': 16, 'leng

## high-level generators

In [151]:
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
print(f"i_{i1} --[{l1}]-> N_{n1} --[{l2}]-> o_{o1}")
showGraph(pkg)

[2024-12-27 14:24:15]in[connect.315] linked I[i1] to 3
[2024-12-27 14:24:15]in[connect.307] linked N[1] to 4
i_i1 --[3]-> N_1 --[4]-> o_2
{'attenuationPerTick': 0.0, 'events': [], 'id': 3, 'length': 1, 'receiverId': 1}
link  i1  to  1  sz: 1
{'attenuationPerTick': 0.0, 'events': [], 'id': 4, 'length': 1, 'receiverId': 2}
link  1  to  2  sz: 1
nx.html


In [168]:
pkg = mc.Package.make_empty()
n1 = pkg.new_neuron(leak=1.0, peak=5.0, receivers=[])
i1 = pkg.new_tape_input("i1", [])
o1 = pkg.new_output("o1")

l1 = pkg.new_link_between(apt=0.0, length=1, src=i1, dst=n1)
l2 = pkg.new_link_between(apt=0.0, length=1, src=n1, dst=o1)
ce = mc.Engine()
ce.core = mc.Core()
ce.core.load(pkg)
ce.tick_offset = 0

df_expected = pd.DataFrame([0.0]*7 + [5.0] + [0.0]*4, columns=['o1_exp'])
df_in = pd.DataFrame([3,0] * 6, columns=['i1'])
out = [0] * 9 + [5, 0]
ticks = 12
df_out = pd.DataFrame()
for t in range(ticks):
    df_out = pd.concat([df_out, ce.run_once(df_in)], ignore_index=True)



[2024-12-27 15:36:48]in[connect.315] linked I[i1] to 3
[2024-12-27 15:36:48]in[connect.307] linked N[1] to 4
[2024-12-27 15:36:48]in[load.72] found input tape
[2024-12-27 15:36:48]in[addPoint.99] store point i1 : r:[3] ex: {} data:{'type': 'tape', 'name': 'i1', 'receivers': [3], 'args': {}}
[2024-12-27 15:36:48]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:48]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:48]in[__process_autoinputs.153] autoinputs made 0 new events
[2024-12-27 15:36:48]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:48]in[makeEvents.115] proces point [i1] : ([3], {})
[2024-12-27 15:36:48]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:48]in[__process_tapes.160] tape container made 1 new events
[2024-12-27 15:36:48]in[__on_e_changed.85] N[1] changed 0.0 --> 0.0 in [onTick.82]
[2024-12-27 15:36:48]in[__process_content.166] active content (4 objects) made 0 new events
[2024-12-27 15:36:48]in[__process_events.1

[2024-12-27 15:36:49]in[__process_autoinputs.153] autoinputs made 0 new events
[2024-12-27 15:36:49]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:49]in[makeEvents.115] proces point [i1] : ([3], {})
[2024-12-27 15:36:49]in[dprint.58] TapeInputsRow h[1], d[s1.dim1]
[2024-12-27 15:36:49]in[__process_tapes.160] tape container made 1 new events
[2024-12-27 15:36:49]in[__on_e_changed.85] N[1] changed 2.0 --> 1.0 in [onTick.82]
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Link.3.bfc4c7ec7e14ef53 LINK_APPLY_EVENT : {'tick': 10, 'amp': 3.0, 'from': 'i1'}
[2024-12-27 15:36:49]in[__process_content.166] active content (4 objects) made 1 new events
[2024-12-27 15:36:49]in[__process_events.177] process p.event i1->Link_3 amp:0
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Link.3.764e00faf6fb076a LINK_IGNORE_SIGNAL : {'tick': 10, 'amp': 0, 'from': 'i1'}
[2024-12-27 15:36:49]in[__process_events.177] process p.event 3->Neuron_1 amp:3.0
[2024-12-27 15:36:49]in[onRobotsEvent.31] @ Neuron.1.

In [169]:
io = pd.concat([df_in, df_out, df_expected], axis=1)
io['error'] = io['o1'].astype(float) - io['o1_exp'].astype(float)
io

,i1,o1,o1_exp,error
0,3,0.0,0.0,0.0
1,0,0.0,0.0,0.0
2,3,0.0,0.0,0.0
3,0,0.0,0.0,0.0
4,3,0.0,0.0,0.0
5,0,0.0,0.0,0.0
6,3,0.0,0.0,0.0
7,0,5.0,5.0,0.0
8,3,0.0,0.0,0.0
9,0,0.0,0.0,0.0


In [175]:
from IPython.display import JSON
from IPython.display import Markdown

spkg = ce.core.dump()
sdata = spkg.dumpJsonStr()

print(json.dumps(json.loads(sdata), indent=2))


[2024-12-27 16:27:11]in[dump.131] save neuron 1
[2024-12-27 16:27:11]in[dump.134] save link 3
[2024-12-27 16:27:11]in[dump.134] save link 4
{
  "meta": {
    "branch": "default",
    "branchSeq": 6,
    "generation": 0,
    "name": "LObjectIdType.Core_PYMGENN_36d4dc58f916cedec273945910524d3cbed81756b023b97e492182c64f55171a.1735316831573.40",
    "parentDelta": "NONE",
    "parentSnapshot": "LObjectIdType.Core_PYMGENN_36d4dc58f916cedec273945910524d3cbed81756b023b97e492182c64f55171a.1735316815224.39",
    "rev": 0,
    "tick": 0
  },
  "state": "new",
  "contenttype": "json",
  "content": {
    "parent": "LObjectIdType.Core_PYMGENN_36d4dc58f916cedec273945910524d3cbed81756b023b97e492182c64f55171a.1735316815224.39",
    "tick": 12,
    "id": "LObjectIdType.Core_PYMGENN_36d4dc58f916cedec273945910524d3cbed81756b023b97e492182c64f55171a.1735316831573.40",
    "generation": 6,
    "seq": 0,
    "inputs": [],
    "outputs": [],
    "storage": {
      "links": [
        {
          "id": "3",
   